In [57]:
import pandas as pd
import numpy as np
import json
import math

import base64
from io import BytesIO
from PIL import Image

from jinja2 import Environment, Template, FileSystemLoader

from sklearn.manifold import TSNE

In [52]:
env = Environment()
env.filters['jsonify'] = json.dumps

In [3]:
df_numeric = pd.read_pickle('data/dataframes/pokemon_numeric.pickle')
df_numeric_scaled = pd.read_pickle('data/dataframes/pokemon_numeric_scaled.pickle')
df_against = pd.read_pickle('data/dataframes/pokemon_against.pickle')
df_names = pd.read_pickle('data/dataframes/pokemon_names.pickle')
df_types = pd.read_pickle('data/dataframes/pokemon_types.pickle')
df_classfication = pd.read_pickle('data/dataframes/pokemon_classfication.pickle')
df_abilities = pd.read_pickle('data/dataframes/pokemon_abilities.pickle')
df_sprites = pd.read_pickle('data/dataframes/pokemon_sprites.pickle.bz2', compression='bz2')

In [109]:
df_features = df_numeric_scaled.merge(df_against, left_index=True, right_index=True).merge(df_types, left_index=True, right_index=True).merge(df_classfication, left_index=True, right_index=True).merge(df_abilities, left_index=True, right_index=True)

In [110]:
df_data = df_numeric.merge(df_names, left_index=True, right_index=True).merge(df_sprites, left_on='pokedex_number', right_on='pokedex_number')

In [111]:
tsne = TSNE(n_components=2, n_iter=300)
tsne_result = tsne.fit_transform(df_features.values)

In [112]:
df_data['tsne_x'] = tsne_result[:,0]
df_data['tsne_y'] = tsne_result[:,1]

In [114]:
limits = {}
limits['xmin'], limits['xmax'] = (math.floor(df_data['tsne_x'].min()), math.ceil(df_data['tsne_x'].max()))
limits['ymin'], limits['ymax'] = (math.floor(df_data['tsne_y'].min()), math.ceil(df_data['tsne_y'].max()))

In [117]:
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f']
generations = []
for gen in df_data['generation'].unique():
    color = colors[gen - 1]
    name = 'Generation {}'.format(gen)
    series = []
    for index, row in df_data[df_data['generation'] == gen].iterrows():
        pil_img = Image.fromarray(row['image_data'])
        pil_img.thumbnail(size=(50,50))
        buff = BytesIO()
        pil_img.save(buff, format="PNG")
        image_string = base64.b64encode(buff.getvalue()).decode("utf-8")
        series.append({
            'x': row['tsne_x'],
            'y': row['tsne_y'],
            'name': row['name'],
            'image_data': image_string
        })
    generations.append({
        'name': name,
        'color': color,
        'data': series
    })

In [123]:
with open('static/highcharts_scatter_2d_template.html') as file_:
    template = file_.read()
    with open('static/highcharts_scatter_2d_gen.html', 'w') as outfile_:
        outfile_.write(env.from_string(template).render(series=generations, limits=limits))